In [1]:
# !pip install -r requirements.txt

In [2]:
cd E:/projects/ft-llm-tp

E:\projects\ft-llm-tp


In [3]:
import sys
sys.path.insert(0, 'E:/projects/ft-llm-tp')

In [4]:
from src.utils.model_utils import load_tokenizer

In [5]:
from utils import load_args

In [22]:
load_args('configs/exp.yaml')

Loaded Args:
exp_manager:
  seed: 202412
  exps_dir: ./exps
  exp_name: it__qwen2.5-0.5b-instruct__cmc_global_qa_480
  prepare_data_cfg_path: configs/prepare_data.yaml
  train_cfg_path: configs/train.yaml
  task_name: null
  model_name: null
  dataset_name: null
wandb:
  use_wandb: true
  project: llm_ft_ask_cmc_global





{'exp_manager': {'seed': 202412, 'exps_dir': './exps', 'exp_name': 'it__qwen2.5-0.5b-instruct__cmc_global_qa_480', 'prepare_data_cfg_path': 'configs/prepare_data.yaml', 'train_cfg_path': 'configs/train.yaml', 'task_name': None, 'model_name': None, 'dataset_name': None}, 'wandb': {'use_wandb': True, 'project': 'llm_ft_ask_cmc_global'}}

In [21]:
prepare_data_args = load_args('configs/prepare_data.yaml')

Loaded Args:
dataset:
  data_path: E:/projects/ask-aimesoft/notebooks/draft/list_qa_results_20241209_024009_fixed_row8_list_type.csv
  do_split: true
  val_ratio: 0.25
  test_ratio: 0.2
  output_path: null
prompt:
  use_model_chat_template: true
  instruction_key: '### Instruction:'
  instruction_text: You are a knowledgeable assistant for the company CMC Global.
    Your task is to providing accurate and helpful answers to the user's questions
    about the company.
  input_key: '### Question:'
  response_key: '### Answer:'
  end_key: null
tokenizer:
  model_name_or_path: Qwen/Qwen2.5-0.5B-Instruct
  new_pad_token: null
  do_tokenize: false
  truncation: true
  padding: max_length
  add_special_tokens: true
  max_length: 128





In [8]:
prepare_data_args.dataset

{'data_path': 'E:/projects/ask-aimesoft/notebooks/draft/list_qa_results_20241209_024009_fixed_row8_list_type.csv', 'do_split': True, 'val_ratio': 0.25, 'test_ratio': 0.2, 'output_path': None}

In [9]:
train_args = load_args('configs/train.yaml')

Loaded Args:
exp_name: it__qwen2.5-0.5b-instruct__cmc_global_qa_480
seed: 202412
model_args:
  pretrained_model_name_or_path: Qwen/Qwen2.5-0.5B-Instruct
  load_in_4bit: true
  load_in_8bit: false
  bnb_4bit_compute_dtype: null
  bnb_4bit_quant_type: nf4
  bnb_4bit_use_double_quant: false
  bnb_4bit_quant_storage: uint8
use_peft: true
lora:
  r: 64
  lora_alpha: 32
  lora_dropout: 0.0
  bias: none
  task_type: CAUSAL_LM
  inference_mode: false
  target_modules:
  - q_proj
  - k_proj
  - v_proj
  - o_proj
  - gate_proj
  - up_proj
  - down_proj
train_args:
  _target_: transformers.TrainingArguments
  resume_from_checkpoint: true
  do_train: true
  do_eval: true
  do_predict: true
  learning_rate: 0.0001
  num_train_epochs: 1
  per_device_train_batch_size: 2
  per_device_eval_batch_size: 2
  logging_steps: 1
  logging_first_step: true
  save_strategy: epoch
  eval_strategy: epoch
  eval_on_start: true





In [10]:
model_args = train_args.model_args

In [11]:
from src.utils.model_utils import load_model, load_tokenizer, get_model_tokenizer, get_peft_config

In [12]:
from huggingface_hub import login
login()

In [13]:
# tokenizer = load_tokenizer(prepare_data_args, model_args)

In [14]:
# moel = load_model(model_args, use_cpu=True)

In [15]:
model, tokenizer = get_model_tokenizer(prepare_data_args, model_args)

In [16]:
model.config

Qwen2Config {
  "_attn_implementation_autoset": true,
  "_name_or_path": "Qwen/Qwen2.5-0.5B-Instruct",
  "architectures": [
    "Qwen2ForCausalLM"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 151643,
  "eos_token_id": 151645,
  "hidden_act": "silu",
  "hidden_size": 896,
  "initializer_range": 0.02,
  "intermediate_size": 4864,
  "max_position_embeddings": 32768,
  "max_window_layers": 21,
  "model_type": "qwen2",
  "num_attention_heads": 14,
  "num_hidden_layers": 24,
  "num_key_value_heads": 2,
  "rms_norm_eps": 1e-06,
  "rope_scaling": null,
  "rope_theta": 1000000.0,
  "sliding_window": null,
  "tie_word_embeddings": true,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.47.1",
  "use_cache": true,
  "use_sliding_window": false,
  "vocab_size": 151665
}

In [17]:
len(tokenizer)

151665

In [18]:
data_path = "E:/projects/ask-aimesoft/notebooks/draft/list_qa_results_20241209_024009_fixed_row8_list_type.csv"

In [19]:
import pandas as pd 
pd.read_csv(data_path)

,qa_id,question_type,question,answer,url
0,company_overview__qa1,company_overview,CMC Global cung cấp giải pháp gì cho khách hàng?,"CMC Global cung cấp giải pháp IT một cửa, bao ...",https://cmcglobal.com.vn/
1,company_overview__qa2,company_overview,Giải pháp IT Outsourcing của CMC Global mang l...,Giải pháp IT Outsourcing của CMC Global giúp c...,https://cmcglobal.com.vn/
2,company_overview__qa3,company_overview,CMC Global hỗ trợ chuyển đổi số như thế nào?,CMC Global hỗ trợ việc áp dụng các công nghệ t...,https://cmcglobal.com.vn/
3,company_overview__qa4,company_overview,Dịch vụ nào thuộc danh mục giải pháp của CMC G...,Các dịch vụ thuộc danh mục giải pháp của CMC G...,https://cmcglobal.com.vn/
4,company_overview__qa5,company_overview,Các giải pháp CMC Global có gì đặc biệt?,Giải pháp của CMC Global bao gồm các dịch vụ c...,https://cmcglobal.com.vn/
...,...,...,...,...,...
475,customer_support__qa16,customer_support,Mọi văn phòng ở Tokyo đều có số điện thoại giố...,"Có, tất cả các văn phòng ở Tokyo đều có số điệ...",https://cmcglobal.com.vn/contact-us/
476,customer_support__qa17,customer_support,Có văn phòng nào ở hải ngoại không?,"Có, công ty có văn phòng tại Tokyo và Singapor...",https://cmcglobal.com.vn/contact-us/
477,customer_support__qa18,customer_support,Công ty có văn phòng nào ở khu vực trung tâm Đ...,"Có, văn phòng ở Đà Nẵng nằm ngay tại trung tâm...",https://cmcglobal.com.vn/contact-us/
478,customer_support__qa19,customer_support,Văn phòng nào có địa chỉ cụ thể tại quận Jung-gu?,Văn phòng ở Seoul có địa chỉ cụ thể tại quận J...,https://cmcglobal.com.vn/contact-us/


In [20]:
from prepare_data import has_system_role_support
has_system_role_support(tokenizer)

True

In [ ]:
!

In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer, PreTrainedTokenizer
from typing import Tuple

def load_tokenizer(data_args, model_args,
                  # padding_side
) -> PreTrainedTokenizer:
    tokenizer =  AutoTokenizer.from_pretrained(model_args.pretrained_model_name_or_path)

    if not tokenizer.pad_token:
        if data_args.new_pad_token:
            tokenizer.padding_side = 'left'
            tokenizer.pad_token = data_args.new_pad_token,
            tokenizer.add_special_tokens({"pad_token": data_args.new_pad_token})
        else:
            tokenizer.padding_side = 'right'
            tokenizer.pad_token = tokenizer.eos_token
            
    return tokenizer

In [26]:
tokenizer = load_tokenizer(prepare_data_args, model_args)
len(tokenizer)

151665

In [28]:
prepare_data_args.prompt.instruction_key

'### Instruction:'

In [31]:
INSTRUCTION_KEY = prepare_data_args.prompt.instruction_key
INPUT_KEY = prepare_data_args.prompt.input_key
RESPONSE_KEY = prepare_data_args.prompt.response_key

In [32]:
tokenizer.add_special_tokens({"additional_special_tokens": [INPUT_KEY, INSTRUCTION_KEY, RESPONSE_KEY]})

3

In [ ]:
len(tokenizer)

151668

In [36]:
tokenizer

Qwen2TokenizerFast(name_or_path='Qwen/Qwen2.5-0.5B-Instruct', vocab_size=151643, model_max_length=131072, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '<|im_end|>', 'pad_token': '<|endoftext|>', 'additional_special_tokens': ['### Question:', '### Instruction:', '### Answer:']}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	151643: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151644: AddedToken("<|im_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151645: AddedToken("<|im_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151646: AddedToken("<|object_ref_start|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151647: AddedToken("<|object_ref_end|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	151648: AddedToken("<|b

In [37]:
len(tokenizer)

151668